In [1]:
#import libraries
import pandas as pd
import numpy as np

np.random.seed(12345)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Loading Raw Data

In [2]:
#loading datasets
df_2018_main = pd.read_csv('data\Citywide_Mobility_Survey_-_Main_Survey_2018.csv', na_values=' ', low_memory=False)
df_2018_trip = pd.read_csv('data\Citywide_Mobility_Survey_-_Trip_Diary_2018.csv', na_values=' ', low_memory=False)

df_2017_main = pd.read_csv('data\Citywide_Mobility_Survey_-_Main_Survey_2017.csv', na_values=' ', low_memory=False)
df_2017_trip = pd.read_csv('data\Citywide_Mobility_Survey_-_Trip_Diary_2017.csv', na_values=' ', low_memory=False)

df_2019_main = pd.read_csv('data\Citywide_Mobility_Survey_-_Person_Survey_2019.csv', na_values=' ', low_memory=False)
df_2019_trip = pd.read_csv('data\Citywide_Mobility_Survey_-_Trip_Survey_2019.csv', na_values=' ', low_memory=False)
df_2019_household = pd.read_csv('data\Citywide_Mobility_Survey_-_Household_Survey_2019.csv', na_values=' ', low_memory=False)

df_2020_july = pd.read_csv('data\Citywide_Mobility_Survey_-_July_2020.csv', na_values=' ', low_memory=False)
df_2020_may = pd.read_csv('data\Citywide_Mobility_Survey_-_May_2020.csv', na_values=' ', low_memory=False)
df_2020_oct = pd.read_csv('data\Citywide_Mobility_Survey_-_October_2020.csv', na_values=' ', low_memory=False)

# Trip Survey Data Cleaning

### selecting columns and aggregating

In [3]:
#cleaning 2017 trip data person id
df_2017_trip[['Respondent_Serial','cfmc_case_id','qSurveyZone','qModeGrouping','qSustainableMode','qday1trippurpose']]
df_2017_trip['Respondent_Serial'] = pd.to_numeric(df_2017_trip['Respondent_Serial'].astype('str') , errors='coerce').convert_dtypes()
df_2017_trip['cfmc_case_id'] =pd.to_numeric(df_2017_trip['cfmc_case_id'].astype('str') , errors='coerce').convert_dtypes()
df_2017_trip['person_id']=np.where(df_2017_trip['Respondent_Serial'].isnull(), df_2017_trip['cfmc_case_id'], df_2017_trip['Respondent_Serial'])

In [30]:
#aggrergating 2017 trip data
trip_2017 = df_2017_trip.groupby(['person_id','qSurveyZone','qday','qModeGrouping','qSustainableMode','qday1trippurpose']).agg({'UniqueID':'count', 'qday1triplength':'sum'}).reset_index().rename(columns={'UniqueID':'num_trips'})
trip_2017.head(3)

,person_id,qSurveyZone,qday,qModeGrouping,qSustainableMode,qday1trippurpose,num_trips,qday1triplength
0,360,8,6,6,2,1,1,15
1,360,8,6,6,2,6,1,15
2,360,8,6,6,2,8,1,3


In [31]:
#aggrergating 2018 trip data
df_2018_trip[['UniqueID','qSURVEYZONE_HOME','qModeGrouping','qSustainableMode','qDAY1TRIPPURPOSE']]
trip_2018 = df_2018_trip.groupby(['UniqueID','qSURVEYZONE_HOME','QDAY','qModeGrouping','qSustainableMode','qDAY1TRIPPURPOSE']).agg({'Trip_ID':'count', 'qDAY1TRIPLENGTH':'sum'}).reset_index().rename(columns={'Trip_ID':'num_trips'})
trip_2018.head(3)

,UniqueID,qSURVEYZONE_HOME,QDAY,qModeGrouping,qSustainableMode,qDAY1TRIPPURPOSE,num_trips,qDAY1TRIPLENGTH
0,100012,Middle Queens,Thursday,Subway,Yes,Commute to/from work,2,120
1,100012,Middle Queens,Thursday,Walk,Yes,Dining,1,10
2,100013,Outer Brooklyn,Thursday,Walk,Yes,Commute to/from work,1,15


In [32]:
#aggrergating 2019 trip data
df_2019_trip[['person_id','home_cms_zone','day_num','mode_type','sustainable_mode','o_purpose_category_imputed']]
trip_2019_wo995 = df_2019_trip[df_2019_trip['mode_type']!=995] 
trip_2019 = trip_2019_wo995.groupby(['person_id','home_cms_zone','day_num','mode_type','sustainable_mode','o_purpose_category_imputed']).agg({'trip_id':'count', 'trip_duration':'sum'}).reset_index().rename(columns={'trip_id':'num_trips'})
trip_2019.head(3)

,person_id,home_cms_zone,day_num,mode_type,sustainable_mode,o_purpose_category_imputed,num_trips,trip_duration
0,1900023501,Northern Bronx,1,6,1,-9998,1,11.10000
1,1900023501,Northern Bronx,3,7,0,2,1,9.10000
2,1900023501,Northern Bronx,4,7,0,-9998,1,17.90000


### cleaning and standardizing trip survey

In [33]:
#renaming 2018
trip_2018_coded = trip_2018.rename(columns={'UniqueID':'person_id','qSURVEYZONE_HOME':'survey_zone','QDAY':'trip_day','qModeGrouping':'trip_mode','qSustainableMode':'sustainable_mode','qDAY1TRIPPURPOSE':'trip_purpose','qDAY1TRIPLENGTH':'trip_duration'})
trip_2018_coded['survey_year']='2018'
trip_2018_coded.head(3)

,person_id,survey_zone,trip_day,trip_mode,sustainable_mode,trip_purpose,num_trips,trip_duration,survey_year
0,100012,Middle Queens,Thursday,Subway,Yes,Commute to/from work,2,120,2018
1,100012,Middle Queens,Thursday,Walk,Yes,Dining,1,10,2018
2,100013,Outer Brooklyn,Thursday,Walk,Yes,Commute to/from work,1,15,2018


In [20]:
#converting values to categorical in trip 2017
t2017_qsurvey_zone = {1: 'Inner Brooklyn' ,2: 'Inner Queens' ,3: 'Manhattan Core' ,4: 'Middle Queens' ,5: 'Northern Bronx' ,6: 'Northern Manhattan' ,7: 'Outer Brooklyn' ,8: 'Outer Queens' ,9: 'Southern Bronx' ,10: 'Staten Island' ,11: 'Other'}
t2017_qday = {1:'Sunday', 2:'Monday', 3:'Tuesday', 4:'Wednesday', 5:'Thursday', 6:'Friday', 7:'Saturday', 8:'Not coded'}
t2017_qmodegroup = {1: 'For-Hire Vehicle' ,2: 'Commuter rail' ,3: 'Bus' ,4: 'Ferry' ,5: 'Subway' ,6: 'Car' ,7: 'Bike' ,8: 'Walk' ,9: 'Other'}
t2017_sustmode = {1: 'Yes', 2:'No'}
t2017_tripp = {1: 'Return home' ,2: 'Commute to work/from' ,3: 'Business' ,4: 'School' ,5: 'Shopping' ,6: 'Social/recreation' ,7: 'Personal errands' ,8: 'Medical visit (doctor’s office)' ,9: 'Accompanying  other traveler' ,10: 'Other' ,11: 'Don’t know' ,12: 'Refused' }


trip_2017_coded = trip_2017.replace({'qSurveyZone':t2017_qsurvey_zone ,
                                    'qday': t2017_qday,
                                    'qModeGrouping': t2017_qmodegroup,
                                    'qSustainableMode':t2017_sustmode,
                                    'qday1trippurpose':t2017_tripp}).rename(columns={'qSurveyZone':'survey_zone','qday':'trip_day','qModeGrouping':'trip_mode','qSustainableMode':'sustainable_mode','qday1trippurpose':'trip_purpose','qday1triplength':'trip_duration'})
trip_2017_coded['survey_year']='2017'
trip_2017_coded.head(3)

,person_id,survey_zone,trip_day,trip_mode,sustainable_mode,trip_purpose,num_trips,trip_duration,survey_year
0,360,Outer Queens,Friday,Car,No,Return home,1,15,2017
1,360,Outer Queens,Friday,Car,No,Social/recreation,1,15,2017
2,360,Outer Queens,Friday,Car,No,Medical visit (doctor’s office),1,3,2017


In [21]:
#converting values to categorical in trip 2019
t2019_qday = { 1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7:'Sunday',995:'Not coded'}
t2019_trip_mode = {1: 'For-Hire Vehicle' ,2: 'For-Hire Vehicle' ,3: 'Commuter Rail' ,4: 'Bus' ,5: 'Ferry' ,6: 'Subway' ,7: 'Car' ,8: 'Bike' ,9: 'Walk' ,10: 'Other' }
t2019_sustmode = {0:'No', 1:'Yes'}
t2019_tripp = {1: 'Home' ,2: 'Work' ,3: 'Work-related' ,4: 'School' ,5: 'Escort' ,6: 'Shop' ,7: 'Meal' ,8: 'Social/Recreation' ,9: 'Errand/Other' ,10: 'Change mode' ,11: 'Other (Unknown)' ,995: 'Missing' ,-9998: 'Non-response' }
trip_2019_coded = trip_2019.replace({'day_num':t2019_qday, 
                                     'mode_type':t2019_trip_mode, 
                                     'sustainable_mode':t2019_sustmode, 
                                     'o_purpose_category_imputed':t2019_tripp}).rename(columns={'home_cms_zone':'survey_zone','day_num':'trip_day','mode_type':'trip_mode', 'o_purpose_category_imputed':'trip_purpose'})
trip_2019_coded['survey_year']='2019'
trip_2019_coded.head(3)

,person_id,survey_zone,trip_day,trip_mode,sustainable_mode,trip_purpose,num_trips,trip_duration,survey_year
0,1900023501,Northern Bronx,Monday,Subway,Yes,Non-response,1,11.10000,2019
1,1900023501,Northern Bronx,Wednesday,Car,No,Work,1,9.10000,2019
2,1900023501,Northern Bronx,Thursday,Car,No,Non-response,1,17.90000,2019


In [22]:
#combines all trip survey data
all_survey_trip_data = pd.concat([trip_2017_coded,trip_2018_coded,trip_2019_coded], ignore_index=True)
all_survey_trip_data.head(3)

,person_id,survey_zone,trip_day,trip_mode,sustainable_mode,trip_purpose,num_trips,trip_duration,survey_year
0,360,Outer Queens,Friday,Car,No,Return home,1,15.00000,2017
1,360,Outer Queens,Friday,Car,No,Social/recreation,1,15.00000,2017
2,360,Outer Queens,Friday,Car,No,Medical visit (doctor’s office),1,3.00000,2017


# Main Survey Data Cleaning

In [24]:
#cleaning 2017 main survey data
df_2017_main['Respondent_Serial'] = pd.to_numeric(df_2017_main['Respondent_Serial'].astype('str') , errors='coerce').convert_dtypes()
df_2017_main['cfmc_case_id'] =pd.to_numeric(df_2017_main['cfmc_case_id'].astype('str') , errors='coerce').convert_dtypes()
df_2017_main['person_id']=np.where(df_2017_main['Respondent_Serial'].isnull(), df_2017_main['cfmc_case_id'], df_2017_main['Respondent_Serial'])
df_2017_main = df_2017_main.dropna(subset=['person_id'])
#df_2017_main[['person_id','qgender' ,'qage' ,'qrace' ,'qrent' ,'qchildren' ,'qincome' ,'qemployment' ,'qDISABILITY1' ,'qDISABILITY2' ,'qDISABILITY3' ,'qDISABILITY4' ,'qDISABILITY5' ,'qDISABILITY6' ,'qDISABILITY7' ,'qDISABILITY8' ,'qeducation' ,'qcarmany' ,'qbikeown']]

In [12]:
#check if any duplicate person id 
sum(df_2017_main.duplicated(subset=['person_id']))

0

In [25]:
#converting numeric to categories
m2017_gender = {1: 'Male' ,2: 'Female' ,3: 'Dont know' ,4: 'Refused' }
m2017_race = {1: 'White/Caucasian' ,2: 'Black/ African American/ Caribbean American' ,3: 'Asian' ,4: 'American Indian or Alaska Native' ,5: 'Native Hawaiian or Pacific Islander' ,6: 'Other' ,7: 'Two or more races' ,8: 'Dont know' ,9: 'Refused' }
m2017_rent = {1: 'Rent' ,2: 'Own' ,3: 'Dont know' ,4: 'Refused' }
m2017_income = {1: 'Less than $14,999' ,2: '$15,000 - $24,999' ,3: '$25,000 - $34,999' ,4: '$35,000 - $49,999' ,5: '$50,000 - $74,999' ,6: '$75,000 - $99,999' ,7: '$100,000 - $149,999' ,8: '$150,000-$199,999' ,9: '$200,000 and above' ,10:' Dont know' ,11:' Refused'}
m2017_employment = {1: 'Employed full time' ,2: 'Employed part time' ,3: 'Unemployed' ,4: 'Stay-at-home parent' ,5: 'Volunteer work' ,6: 'Full time student' ,7: 'Part-time student' ,8: 'Retired' ,9: 'Active military service' ,10: 'Other' ,11: 'Dont know' ,12: 'Refused' }
m2017_yesno = {0:'No', 1:'Yes'}
m2017_educ = {1: 'No high school' ,2: 'Some high school' ,3: 'High school graduate or equivalent' ,4: 'Some college but degree not received or in progress' ,5: 'Associate degree' ,6: 'Bachelor\'s degree' ,7: 'Graduate degree' ,8: 'Don’t know' ,9: 'Refused' }
m2017_children = {1: 'None' ,2: 'One' ,3: 'Two' ,4: 'Three' ,5: 'Four or more' ,6: 'Dont know' ,7: 'Refused' }

df_2017_main['has_disability'] = np.where((df_2017_main.iloc[:,329:334].sum(axis=1))>0,'Yes','No')
df_2017_main['has_caraccess'] = np.where(df_2017_main.qcarmany>0, 'Yes','No')
df_2017_main['has_bike'] = np.where(df_2017_main.qbikeown>0, 'Yes','No')

df_2017_main_coded = df_2017_main[['person_id','qgender' ,'qage' ,'qrace' ,'qrent' 
                    ,'qchildren' ,'qincome' ,'qemployment' , 'qeducation'
                    , 'has_disability', 'has_caraccess','has_bike']].replace({'qgender':m2017_gender, 
                                           'qrace':m2017_race, 
                                           'qrent':m2017_rent, 
                                           'qincome':m2017_income, 
                                           'qemployment':m2017_employment,
                                           'qeducation':m2017_educ,
                                            'qchildren':m2017_children,
                                          }).rename(columns={'qgender':'gender', 'qage':'age',
                                                                            'qrace':'race', 'qrent':'rent', 'qchildren':'num_children',
                                                                            'qincome':'income','qemployment':'employment', 'qeducation':'education',
                                                                            })
df_2017_main_coded['survey_year'] = '2017'
df_2017_main_coded.head(3)

,person_id,gender,age,race,rent,num_children,income,employment,education,has_disability,has_caraccess,has_bike,survey_year
0,10000001,Female,68.00000,White/Caucasian,Own,None,Refused,Retired,Graduate degree,No,No,Yes,2017
1,10000002,Female,999.00000,White/Caucasian,Own,None,"$35,000 - $49,999",Retired,Graduate degree,Yes,No,Yes,2017
2,10000008,Female,41.00000,Two or more races,Own,One,"$50,000 - $74,999",Employed part time,Some high school,No,No,Yes,2017


In [18]:
#any duplicate unique id of respondents
sum(df_2018_main.duplicated(subset='UniqueID'))

0

In [26]:
#cleaning 2018 main survey
df_2018_main['has_disability'] = np.where(df_2018_main.iloc[:,175:181].replace({'Yes':True, 'No':False}).any(axis=1),'Yes','No')
df_2018_main['qEMPLOYMENT'].replace({'Employed full time (working more than 30 hours per week)':'Employed full time'
                                    ,'Employed part time (working less than 30 hours per week)':'Employed part time'
                                    ,'Volunteer work (at least 1 day/week)':'Volunteer work'},inplace=True)
df_2018_main['qEDUCATION'].replace({'Associate degree (i.e., AA, AS)':'Associate degree',
                                   'Bachelor\'s degree (i.e., BA, BS, AB)':'Bachelor\'s degree',
                                   'Graduate degree (i.e., Master\'s, Professional, Doctorate)':'Graduate degree',
                                   'High school graduate or equivalent (i.e., GED)':'High school graduate or equivalent'}
                                  ,inplace=True)

df_2018_main_coded = df_2018_main[['UniqueID','qGENDER','qAGE','qRACE','qRENT','qCHILDREN','qINCOME','qEMPLOYMENT','qEDUCATION',
                                  'has_disability','qCARACCESS','qBIKEOWN']].rename(columns={'UniqueID':'person_id','qGENDER':'gender', 
                                                                                             'qAGE':'age','qRACE':'race', 'qRENT':'rent'
                                                                                             , 'qCHILDREN':'num_children','qINCOME':'income'
                                                                                             ,'qEMPLOYMENT':'employment', 'qEDUCATION':'education'
                                                                                             ,'qCARACCESS':'has_caraccess', 'qBIKEOWN':'has_bike'
                                                                            })
df_2018_main_coded.head(3)

,person_id,gender,age,race,rent,num_children,income,employment,education,has_disability,has_caraccess,has_bike
0,100938,Female,38,White/Caucasian,Own,None,"$150,000-$199,999",Employed full time,Graduate degree,No,"I do not personally own or lease a car, but I ...",Yes
1,105388,Male,55,White/Caucasian,Own,None,"$100,000 - $149,999",Unemployed,Bachelor's degree,No,I do not have access to a car,Yes
2,105415,Male,65,White/Caucasian,Rent,None,"$25,000 - $34,999",Retired,Bachelor's degree,No,I do not have access to a car,No


In [19]:
#any duplicate unique id of respondents
sum(df_2019_main.duplicated(subset='person_id'))

0

In [29]:
#cleaning 2019 main survey data - person + household
m2019_gender = {1: 'Female' ,2: 'Male' ,4: 'Non-binary' ,995: 'Missing' ,997: 'Other/Prefer to self-describe' ,999: 'Prefer not to answer' ,-9998: 'Non-response' }
m2019_employment = {1: 'Employed full-time' ,2: 'Employed part-time' ,3: 'Primarily self-employed' ,6: 'Not currently employed' ,7: 'Unpaid volunteer or intern' ,995: 'Missing: Skip logic' ,-9998: 'Missing: Non-response'}


df_2019_main[['person_id','gender','age','race_american_indian', 'race_asian', 'race_black'
              , 'race_native_hawaiian', 'race_white', 'race_other', 'employment','student'
              ,'disability_hearing','disability_seeing','disability_walking','disability_mobility'
              ,'disability_dressing' ,'disability_concentrating' ,'disability_none','education']].head(3)



,person_id,gender,age,race_american_indian,race_asian,race_black,race_native_hawaiian,race_white,race_other,employment,student,disability_hearing,disability_seeing,disability_walking,disability_mobility,disability_dressing,disability_concentrating,disability_none,education
0,19154632203,995,9,995,995,995,995,995,995,3,0,995,995,995,995,995,995,995,995
1,19154632202,995,8,995,995,995,995,995,995,6,0,995,995,995,995,995,995,995,995
2,19154632201,1,5,0,0,0,1,0,0,6,1,0,0,0,0,0,0,1,5
